In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chama

In [2]:
#Define search area boundary as dict()
boundary = {"min x":0,
            "min y": 0,
            "max x": 300,
            "max y": 30,
            "min z":0,
            "max z": 20
            }

grid_distance = 10

node_x = int((boundary["max x"] - boundary["min x"])/grid_distance)+1
node_y = int((boundary["max y"] - boundary["min y"])/grid_distance)+1
node_z = 21
# Create the Simulation gridspace
x_grid = np.linspace(0, 200, 41)
y_grid = np.linspace(boundary["min y"], boundary["max y"], node_y)
y_grid = np.array([10])
z_grid = np.linspace(boundary["min z"], boundary["max z"], node_z)
z_grid = np.array([1, 2, 3, 5, 10, 15])
grid = chama.simulation.Grid(x_grid, y_grid, z_grid)

In [3]:
# Values 
wind_speeds = [3,10]
Stability_classes = ['A','B','C','D','E','F']
heights = [1,10]
source_rates = np.array([0.1, 0.5, 1.0, 10.0, 25.0])/3600 #

In [4]:
#Low Wind (3m/s)
# all release rates
# both heights
scenario_num = 1
Source_position_1 = [0, 10, heights[0], source_rates[0]]
source_1 = chama.simulation.Source(Source_position_1[0], Source_position_1[1], Source_position_1[2], Source_position_1[3])

atm = pd.DataFrame({'Wind Direction': [0],
            'Wind Speed': [wind_speeds[0]],
              'Stability Class': [Stability_classes[0]]}, index=[0])
gauss_plume = chama.simulation.GaussianPlume(grid, source_1, atm, density_eff = 0.664)
signalGuassPlume = gauss_plume.conc
column_name = f'S {scenario_num}'
print(signalGuassPlume.head())
signalGuassPlume = signalGuassPlume.rename(columns={'S':column_name})
print(signalGuassPlume.head())
signalGuassPlume.insert(4, 'S_Class', Stability_classes[0])
signalGuassPlume.insert(5, 'W_Speed', wind_speeds[0])
signalGuassPlume.insert(6, 'R_rate', source_rates[0])
signalGuassPlume.insert(7, 'H', heights[0])
print(signalGuassPlume.head())
signal = signalGuassPlume


     X     Y     Z  T    S
0  0.0  10.0   1.0  0  0.0
1  0.0  10.0   2.0  0  0.0
2  0.0  10.0   3.0  0  0.0
3  0.0  10.0   5.0  0  0.0
4  0.0  10.0  10.0  0  0.0
     X     Y     Z  T  S 1
0  0.0  10.0   1.0  0  0.0
1  0.0  10.0   2.0  0  0.0
2  0.0  10.0   3.0  0  0.0
3  0.0  10.0   5.0  0  0.0
4  0.0  10.0  10.0  0  0.0
     X     Y     Z  T S_Class  W_Speed    R_rate  H  S 1
0  0.0  10.0   1.0  0       A        3  0.000028  1  0.0
1  0.0  10.0   2.0  0       A        3  0.000028  1  0.0
2  0.0  10.0   3.0  0       A        3  0.000028  1  0.0
3  0.0  10.0   5.0  0       A        3  0.000028  1  0.0
4  0.0  10.0  10.0  0       A        3  0.000028  1  0.0


In [5]:
signalGuassPlume

,X,Y,Z,T,S_Class,W_Speed,R_rate,H,S 1
0,5.0,10.0,1.0,0,A,3,0.000028,1,2.275001e-06
1,5.0,10.0,2.0,0,A,3,0.000028,1,4.352953e-07
2,5.0,10.0,3.0,0,A,3,0.000028,1,1.928028e-09
3,5.0,10.0,5.0,0,A,3,0.000028,1,4.687189e-19
4,5.0,10.0,10.0,0,A,3,0.000028,1,1.028164e-71
5,5.0,10.0,15.0,0,A,3,0.000028,1,2.902120e-165
6,10.0,10.0,1.0,0,A,3,0.000028,1,6.395068e-07
7,10.0,10.0,2.0,0,A,3,0.000028,1,3.962165e-07
8,10.0,10.0,3.0,0,A,3,0.000028,1,1.069790e-07
9,10.0,10.0,5.0,0,A,3,0.000028,1,5.038441e-10


In [5]:
signal = signal.rename(columns={'S 1':column_name})


In [35]:
s_class = 'A'
# Values 
wind_speeds = [3,10]
Stability_classes = ['A','D','F']
heights = [1,5,10]
source_rates = np.array([10.0, 25.0])/3600 #

conversion = 1416000
scenario_num = 1

for source_rate in source_rates:
    for s_class in Stability_classes:
        for Sensor_h  in [1,5,10]:
            fig = plt.figure()
            i = 111
            ax = plt.subplot(111)
            for height in heights:
                for wind_speed in wind_speeds:
                    Source_position_1 = [0, 10, height, source_rate]
                    source_1 = chama.simulation.Source(Source_position_1[0], Source_position_1[1], Source_position_1[2], Source_position_1[3])

                    atm = pd.DataFrame({'Wind Direction': [0],
                                'Wind Speed': [wind_speed],
                                  'Stability Class': [s_class]}, index=[0])
                    gauss_plume = chama.simulation.GaussianPlume(grid, source_1, atm, density_eff = 0.664)
                    signalGuassPlume = gauss_plume.conc
                    column_name = f'U {wind_speed} H {height}'
                    signalGuassPlume = signalGuassPlume.rename(columns={'S':column_name})
                    signalGuassPlume.insert(4, 'S_Class', s_class)
                    signalGuassPlume.insert(5, 'W_Speed', wind_speed)
                    signalGuassPlume.insert(6, 'R_rate', source_rate)
                    signalGuassPlume.insert(7, 'H', heights[0])
                    signal = signalGuassPlume.loc[signalGuassPlume['Z'] == Sensor_h]
                    signal = signal[column_name]*conversion
                    title = f'Sensor Height: {Sensor_h} (m), S Class: {s_class}, Release rate: {source_rate*3600} (kg/hr)'
                    logg = True
                    if wind_speed == 3: L_color = 'r'
                    else: L_color = 'b'
                    y_lim = (0.1,120)
                    if height == 1:
                        ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    logy = logg,
                                    color = L_color,
                                    ylim = y_lim,
                                    xlim = (10,200),
                                    linestyle = 'dotted')
                    elif height == 5:
                        ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    logy = logg,
                                    color = L_color,
                                    ylim = y_lim,
                                    xlim = (10,200),
                                    linestyle = 'dashed')
                    else:
                        ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    color = L_color,
                                    ylim = y_lim,
                                    xlim = (10,200),
                                    logy = logg)
            x = np.arange(10,200,10)
            a = np.ones(len(x))
            y1 = a*50
            y2 = a*20
            y3 = a*1

            ax.plot(x, y1, alpha=0.7,label='NN 50 ppm')
            ax.plot(x, y2, alpha=0.7,label='SGX 20 ppm')
            ax.plot(x, y3, alpha=0.7,label='TDLAS  1 ppm')
            
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

            # Put a legend to the right of the current axis
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

            fname = f'Log Concentration plot S Class {s_class} Sensor_h {Sensor_h} R {source_rate*3600}.png'
            plt.savefig(fname)
            plt.clf()
            plt.close()


In [11]:
signalGuassPlume

,X,Y,Z,T,S_Class,W_Speed,R_rate,H,U 10 H 10
0,0.0,10.0,1.0,0,F,10,0.006944,1,0.000000e+00
1,0.0,10.0,2.0,0,F,10,0.006944,1,0.000000e+00
2,0.0,10.0,3.0,0,F,10,0.006944,1,0.000000e+00
3,0.0,10.0,5.0,0,F,10,0.006944,1,0.000000e+00
4,0.0,10.0,10.0,0,F,10,0.006944,1,0.000000e+00
...,...,...,...,...,...,...,...,...,...
241,200.0,10.0,2.0,0,F,10,0.006944,1,2.444232e-07
242,200.0,10.0,3.0,0,F,10,0.006944,1,4.152951e-07
243,200.0,10.0,5.0,0,F,10,0.006944,1,1.057939e-06
244,200.0,10.0,10.0,0,F,10,0.006944,1,3.651782e-06


In [8]:
signal

4         0.000000
10     4442.559510
16     1364.491552
22      660.138921
28      390.012410
34      258.035337
40      183.656493
46      137.572469
52      107.021242
58       85.712208
64       70.249581
70       58.667771
76       49.764279
82       42.769553
88       37.172351
94       32.622146
100      28.872028
106      25.743960
112      23.106922
118      20.862735
124      18.936648
130      17.270923
136      15.820388
142      14.549282
148      13.428999
154      12.436430
160      11.552741
166      10.762459
172      10.052770
178       9.412995
184       8.834171
190       8.308731
196       7.830253
202       7.393255
208       6.993035
214       6.625541
220       6.287267
226       5.975167
232       5.686580
238       5.419181
244       5.170923
Name: U 10 H 10, dtype: float64